# Training eines Neuronalen Netzes


## Aufbereitung der Daten


In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
import tensorflow as tf
from tensorflow import keras #loss = keras.losses.mae(price, predictions)


In [3]:
app_df = pd.read_csv('application_record.csv')
app_df.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [4]:
app_df.shape

(438557, 18)

In [5]:
credit_df = pd.read_csv('credit_record.csv')
credit_df.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [6]:
credit_df.shape

(1048575, 3)

## Data Merging


In [7]:
#sorting IDs
app_df = app_df.sort_values('ID')
credit_df = credit_df.sort_values('ID')
#Map the status to bad or good
credit_df['STATUS_BINARY'] = credit_df['STATUS']
credit_df = credit_df.replace({'STATUS_BINARY' :{'C' : 'Good','X' : 'Good','0' : 'Bad', '1' : 'Bad','2' : 'Bad','3' : 'Bad','4' : 'Bad','5' : 'Bad'}}) #C und X sind Good, alle Zahlen 0-5 sind Schlecht
result_df = credit_df.value_counts(subset=['ID', 'STATUS_BINARY']).unstack(fill_value=0).reset_index() #für jede ID wird gezählt, wie viele Good und Bad Einträge es gibt
#decide the user is good or bad based on his/her history majority
result_df.loc[(result_df['Good'] / result_df['Bad'] >= 1), 'Status'] = 1 #Wenn eine ID mehr Good als Bad hat, wird der Gesamtstatus auf 1 gut gesetzt, sonst auf 0 schlecht
result_df.loc[(result_df['Good'] / result_df['Bad'] < 1 ), 'Status'] = 0
result_df['Status'] = result_df['Status'].astype(int)
df = app_df.merge(result_df, how='inner', on=['ID']) #Die zusammengefassten Kreditinformationen werden mit den Antragsdaten durch ID verknüpft
#define the rate of good debts for each user
df['rate (Good/Bad)'] = (df['Good']+0.1) / (df['Bad']+0.1) #Spalte Rate wird erstellt, die das Verhältnis von Good zu Bad für jede ID darstellt. Wahrscheinlich wird +0.1 hinzugefügt, um Division durch 0 zu vermeiden
df.drop(['Good','Bad'],axis=1,inplace = True) #Spalten Good und Bad werden gelöscht
#Renaming columns to more friendly names
df.columns = ['ID', 'Gender', 'Car', 'Realty', 'Children', 'Income', 'Income_Type',
                        'Education_Type', 'Family_Status', 'Housing_Type',  'Age', 'Years_Experience' , 'Mobile_Phone', 'Work_Phone', 
                        'Phone', 'Email', 'Job_Title', 'Total_Family' , 'Status','rate (Good/Bad)']
df.head(5)

,ID,Gender,Car,Realty,Children,Income,Income_Type,Education_Type,Family_Status,Housing_Type,Age,Years_Experience,Mobile_Phone,Work_Phone,Phone,Email,Job_Title,Total_Family,Status,rate (Good/Bad)
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1,6.714286
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,1,6.238095
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,1,3.253521
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,1,1.476190
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,1,51.000000


## Handling Missing Values


In [8]:
df.isna().sum()

ID                      0
Gender                  0
Car                     0
Realty                  0
Children                0
Income                  0
Income_Type             0
Education_Type          0
Family_Status           0
Housing_Type            0
Age                     0
Years_Experience        0
Mobile_Phone            0
Work_Phone              0
Phone                   0
Email                   0
Job_Title           11323
Total_Family            0
Status                  0
rate (Good/Bad)         0
dtype: int64

In [9]:
df.dropna()

,ID,Gender,Car,Realty,Children,Income,Income_Type,Education_Type,Family_Status,Housing_Type,Age,Years_Experience,Mobile_Phone,Work_Phone,Phone,Email,Job_Title,Total_Family,Status,rate (Good/Bad)
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,1,3.253521
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,1,1.476190
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,1,51.000000
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,1,3.459016
6,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,1,5.426230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5150482,F,Y,Y,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10808,-1739,1,0,0,0,Core staff,3.0,0,0.504132
36453,5150483,F,Y,Y,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10808,-1739,1,0,0,0,Core staff,3.0,1,181.000000
36454,5150484,F,Y,Y,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10808,-1739,1,0,0,0,Core staff,3.0,0,0.090909
36455,5150485,F,Y,Y,1,157500.0,Working,Secondary / secondary special,Married,House / apartment,-10808,-1739,1,0,0,0,Core staff,3.0,0,0.047619


## Convert Age and Years of Experience to year format


In [10]:
df['Age'] = df['Age']/(-1*365)
df['Years_Experience'] = df['Years_Experience']/(-1*365)
df.head(10)

,ID,Gender,Car,Realty,Children,Income,Income_Type,Education_Type,Family_Status,Housing_Type,Age,Years_Experience,Mobile_Phone,Work_Phone,Phone,Email,Job_Title,Total_Family,Status,rate (Good/Bad)
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.890411,12.443836,1,1,0,0,NaN,2.0,1,6.714286
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.890411,12.443836,1,1,0,0,NaN,2.0,1,6.238095
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,58.832877,3.106849,1,0,0,0,Security staff,2.0,1,3.253521
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,Sales staff,1.0,1,1.476190
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,Sales staff,1.0,1,51.000000
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,Sales staff,1.0,1,3.459016
6,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,52.356164,8.358904,1,0,1,1,Sales staff,1.0,1,5.426230
7,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,61.545205,-1000.665753,1,0,0,0,NaN,1.0,0,0.219858
8,5008813,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,61.545205,-1000.665753,1,0,0,0,NaN,1.0,0,0.219858
9,5008814,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,61.545205,-1000.665753,1,0,0,0,NaN,1.0,0,0.219858


## Behandlung Kategoriale Daten

Neuronale Netze können kategoriale Daten nicht direkt verbinden, sie müssen durch One-Hot-Encoding in eine numerische Darstellung umgewandelt werden


In [21]:
Cat_features = ['Realty', 'Gender','Car','Income_Type','Education_Type','Family_Status','Housing_Type','Job_Title']

for col in Cat_features: #Einblick in alle Kategorien, die in den Datenstehen
    print(col + ':', df[col].unique())

df.info()

Realty: ['Y' 'N']
Gender: ['M' 'F']
Car: ['Y' 'N']
Income_Type: ['Working' 'Commercial associate' 'Pensioner' 'State servant' 'Student']
Education_Type: ['Higher education' 'Secondary / secondary special' 'Incomplete higher'
 'Lower secondary' 'Academic degree']
Family_Status: ['Civil marriage' 'Married' 'Single / not married' 'Separated' 'Widow']
Housing_Type: ['Rented apartment' 'House / apartment' 'Municipal apartment'
 'With parents' 'Co-op apartment' 'Office apartment']
Job_Title: [nan 'Security staff' 'Sales staff' 'Accountants' 'Laborers' 'Managers'
 'Drivers' 'Core staff' 'High skill tech staff' 'Cleaning staff'
 'Private service staff' 'Cooking staff' 'Low-skill Laborers'
 'Medicine staff' 'Secretaries' 'Waiters/barmen staff' 'HR staff'
 'Realty agents' 'IT staff']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID     

In [ ]:
df_encoded = pd.get_dummies(df, columns=Cat_features, drop_first=True) #getDummies konvertiert so viele 0/1 Variabel als es verschiedene Values gibt, setzte drop_first = True damit keine doppelten oder überflüssigen Informationsspalten gibt

df_encoded.head(10)

,ID,Children,Income,Age,Years_Experience,Mobile_Phone,Work_Phone,Phone,Email,Total_Family,...,Job_Title_Laborers,Job_Title_Low-skill Laborers,Job_Title_Managers,Job_Title_Medicine staff,Job_Title_Private service staff,Job_Title_Realty agents,Job_Title_Sales staff,Job_Title_Secretaries,Job_Title_Security staff,Job_Title_Waiters/barmen staff
0,5008804,0,427500.0,32.890411,12.443836,1,1,0,0,2.0,...,False,False,False,False,False,False,False,False,False,False
1,5008805,0,427500.0,32.890411,12.443836,1,1,0,0,2.0,...,False,False,False,False,False,False,False,False,False,False
2,5008806,0,112500.0,58.832877,3.106849,1,0,0,0,2.0,...,False,False,False,False,False,False,False,False,True,False
3,5008808,0,270000.0,52.356164,8.358904,1,0,1,1,1.0,...,False,False,False,False,False,False,True,False,False,False
4,5008809,0,270000.0,52.356164,8.358904,1,0,1,1,1.0,...,False,False,False,False,False,False,True,False,False,False
5,5008810,0,270000.0,52.356164,8.358904,1,0,1,1,1.0,...,False,False,False,False,False,False,True,False,False,False
6,5008811,0,270000.0,52.356164,8.358904,1,0,1,1,1.0,...,False,False,False,False,False,False,True,False,False,False
7,5008812,0,283500.0,61.545205,-1000.665753,1,0,0,0,1.0,...,False,False,False,False,False,False,False,False,False,False
8,5008813,0,283500.0,61.545205,-1000.665753,1,0,0,0,1.0,...,False,False,False,False,False,False,False,False,False,False
9,5008814,0,283500.0,61.545205,-1000.665753,1,0,0,0,1.0,...,False,False,False,False,False,False,False,False,False,False


In [25]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 49 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   ID                                            36457 non-null  int64  
 1   Children                                      36457 non-null  int64  
 2   Income                                        36457 non-null  float64
 3   Age                                           36457 non-null  float64
 4   Years_Experience                              36457 non-null  float64
 5   Mobile_Phone                                  36457 non-null  int64  
 6   Work_Phone                                    36457 non-null  int64  
 7   Phone                                         36457 non-null  int64  
 8   Email                                         36457 non-null  int64  
 9   Total_Family                                  36457 non-null 

In [26]:
df_encoded.describe()

,ID,Children,Income,Age,Years_Experience,Mobile_Phone,Work_Phone,Phone,Email,Total_Family,Status,rate (Good/Bad)
count,3.645700e+04,36457.000000,3.645700e+04,36457.000000,36457.000000,36457.0,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000
mean,5.078227e+06,0.430315,1.866857e+05,43.767598,-162.364207,1.0,0.225526,0.294813,0.089722,2.198453,0.554160,28.035957
std,4.187524e+04,0.742367,1.017892e+05,11.508356,377.126945,0.0,0.417934,0.455965,0.285787,0.911686,0.497065,87.146309
min,5.008804e+06,0.000000,2.700000e+04,20.517808,-1000.665753,1.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.001637
25%,5.042028e+06,0.000000,1.215000e+05,34.142466,1.117808,1.0,0.000000,0.000000,0.000000,2.000000,0.000000,0.192547
50%,5.074614e+06,0.000000,1.575000e+05,42.638356,4.252055,1.0,0.000000,0.000000,0.000000,2.000000,1.000000,1.196078
75%,5.115396e+06,1.000000,2.250000e+05,53.254795,8.638356,1.0,0.000000,1.000000,0.000000,3.000000,1.000000,4.770492
max,5.150487e+06,19.000000,1.575000e+06,68.909589,43.049315,1.0,1.000000,1.000000,1.000000,20.000000,1.000000,611.000000


## Definition von Features (X) und Zielvariable (y)


In [30]:
X = df_encoded.drop(['ID', 'Status'], axis=1) #ID und Status sind keine Features
y = df_encoded['Status'] #Zielvariable

#Aufteilung der Daten in Trainings- und Testsets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50, stratify=y) #stratify=y Versuche die prozentuale Verteilung der Klassen in y_train und y_test so ähnlich wie möglich zu halten wie in y

#Skalierung der numerischen Spalten (Ursprüngliche numerische Spalten in df, die nicht entfernt wurden)

numeric_features = ['Children', 'Income', 'Age', 'Years_Experience', 'Total_Family', 'rate (Good/Bad)']

scaler = StandardScaler()

#Numerische Spalten im Trainingsset fitten und transformieren
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])

# Die gleichen Statistiken des Trainingsset verwenden um das Testset zu transformieren
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

print("Shape of X_train after scalar preprocessing:",  X_train.shape)
print("\nShape of X_test after scalar preprocessing:",X_test.shape)

Shape of X_train after scalar preprocessing: (29165, 47)

Shape of X_test after scalar preprocessing: (7292, 47)


Modellbau, Training und Evaluierung des Neuronalen Netzwerks


Kompilieren des Modells mit Optimierer, Loss-Funktionen und Metriken wie Accuracy


Training des Modelles durch Wiederholung der Trainingsdaten: Nutzung der Epochen und Batches als auch Validierung


Evaluation mithilfe von Loss und Accuracy


Vorhersage treffen auf neue Daten


# Federated Machine Learning
